In [2]:
# -*- coding:utf-8 -*-
# !/usr/bin/python
# Author: Selvaria

# 2维卷积神经网络

import tensorflow as tf
import numpy as np
print(tf.__version__)

2.0.0


In [3]:
# 卷积层中使用的互相关（cross-correlation）运算

def corr2d(X, K):
    h, w = K.shape
    Y = tf.Variable(tf.zeros((X.shape[0] - h + 1, X.shape[1] - w +1)))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            #下面一行解释一下：
            #1.这里最好使用cast函数强制转换一下类型
            #2.assign是tf结构用的赋值函数，不能用等号赋值
            #3.tf.reduce_sum()方法不指定轴会一直累加到只有1个元素，即实现上文的计算
            Y[i,j].assign(tf.cast(tf.reduce_sum(X[i:i+h, j:j+w] * K), dtype=tf.float32)) 
#             Y[i,j] = tf.cast(tf.reduce_sum(X[i:i+h, j:j+w] * K), dtype=tf.float32)
    return Y

X = tf.constant([[0,1,2], [3,4,5], [6,7,8]])
K = tf.constant([[0,1], [2,3]])
corr2d(X, K)

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[19., 25.],
       [37., 43.]], dtype=float32)>

In [1]:
import tensorflow as tf

X = tf.constant([[0,2], [3,5]])
K = tf.constant([[0,1], [2,3]])
r = X*K
r2 = tf.reduce_sum(r, axis=0)
r3 = tf.reduce_sum(r)
print(r)
print(r2)
print(r3)
print(r.shape)

D:\Selvaria\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


tf.Tensor(
[[ 0  2]
 [ 6 15]], shape=(2, 2), dtype=int32)
tf.Tensor([ 6 17], shape=(2,), dtype=int32)
tf.Tensor(23, shape=(), dtype=int32)
(2, 2)


In [4]:
# 卷积层的实现

class Conv2D_self(tf.keras.layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.units = units

    def build(self, kernel_size):
        self.w = self.add_weight(name='w',
                                shape=kernel_size,
                                initializer=tf.random_normal_initializer())
        self.b = self.add_weight(name='b',
                                shape=(1,),
                                initializer=tf.random_normal_initializer())
    def call(self, inputs):
        return corr2d(inputs, self.w) + self.b

In [5]:
import numpy as np

X = np.ones((6,8))
X[:, 2:6] = 0
X

array([[1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.]])

In [9]:
# 简单应用：检测图像中物体的边缘，即找到像素变化的位置。首先我们构造一张6×8的图像。它中间4列为黑（0），其余为白（1）

X = tf.Variable(tf.ones((6,8)))
X[:, 2:6].assign(tf.cast(tf.zeros((6,4)), dtype=tf.float32)) #注意形状作为参数传入要写到元组里
# X[:, 2:6].assign(tf.zeros(X[:,2:6].shape))
X

<tf.Variable 'Variable:0' shape=(6, 8) dtype=float32, numpy=
array([[1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.]], dtype=float32)>

In [6]:
# 设定卷积核

#这个卷积核的特点是，如果扫到原输入的两个元素一样，就会输出零，两元素不同则不为零，从而实现检测边界的目的
K = tf.constant([[1,-1]], dtype = tf.float32)
Y = corr2d(X, K)
Y

<tf.Variable 'Variable:0' shape=(6, 7) dtype=float32, numpy=
array([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.]], dtype=float32)>

In [16]:
# 利用模型让其学习到上个cell我们构造的核数组K

X = tf.reshape(X, (1,6,8,1))
Y = tf.reshape(Y, (1,6,7,1))
# Y

#input_shape = (samples, rows, cols, channels)
conv2d = tf.keras.layers.Conv2D(1, (1,2))
Y.shape

TensorShape([1, 6, 7, 1])

In [19]:
Y_hat = conv2d(X)
print(conv2d.weights)
conv2d.get_weights()
#Y_hat

[<tf.Variable 'conv2d/kernel:0' shape=(1, 2, 1, 1) dtype=float32, numpy=
array([[[[ 0.18774903]],

        [[-0.77540386]]]], dtype=float32)>, <tf.Variable 'conv2d/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


[array([[[[ 0.18774903]],
 
         [[-0.77540386]]]], dtype=float32), array([0.], dtype=float32)]

In [20]:
# 使用tf实现，这里的权重只更新了权重（w），没更新偏置（b一直是0）

for i in range(10):
    with tf.GradientTape(watch_accessed_variables=False) as g:
        g.watch(conv2d.weights[0])
        Y_hat = conv2d(X)
        l = (abs(Y_hat - Y)) ** 2
        dl = g.gradient(l, conv2d.weights[0])
#         print('this is b:',conv2d.weights[1])
        lr = 3e-2
        update = tf.multiply(lr, dl) # 学习率×梯度
        updated_weights = conv2d.get_weights()
        updated_weights[0] = conv2d.weights[0] - update
        conv2d.set_weights(updated_weights)  

        if (i + 1)% 2 == 0:
            print('batch %d, loss %.3f' % (i + 1, tf.reduce_sum(l)))

this is b: <tf.Variable 'conv2d/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>
this is b: <tf.Variable 'conv2d/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>
batch 2, loss 4.636
this is b: <tf.Variable 'conv2d/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>
this is b: <tf.Variable 'conv2d/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>
batch 4, loss 1.580
this is b: <tf.Variable 'conv2d/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>
this is b: <tf.Variable 'conv2d/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>
batch 6, loss 0.593
this is b: <tf.Variable 'conv2d/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>
this is b: <tf.Variable 'conv2d/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>
batch 8, loss 0.234
this is b: <tf.Variable 'conv2d/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>
this is b: <tf.Varia

In [21]:
# 查看学习到的核数组K

tf.reshape(conv2d.get_weights()[0],shape=(1,2))

<tf.Tensor: id=949, shape=(1, 2), dtype=float32, numpy=array([[ 0.9624735, -1.0255724]], dtype=float32)>

In [4]:
# 多通道

# 首先重新构造卷积运算，加入填充规则
def corr2d(X, K):
    h, w = K.shape
    if len(X.shape) <= 1:
        X = tf.reshape(X, (X.shape[0],1))
    Y = tf.Variable(tf.zeros((X.shape[0] - h + 1, X.shape[1] - w +1))) #填充后的卷积层形状
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j].assign(tf.cast(tf.reduce_sum(X[i:i+h, j:j+w] * K), dtype=tf.float32))
    return Y

# 多通道的卷积就是在每个通道做卷积然后累加，这里默认输入的X第一个shape参数为通道
def corr2d_multi_in(X, K):
    return tf.reduce_sum([corr2d(X[i], K[i]) for i in range(X.shape[0])],axis=0)

# 测试
X = tf.constant([[[0,1,2],[3,4,5],[6,7,8]],
                 [[1,2,3],[4,5,6],[7,8,9]]])
K = tf.constant([[[0,1],[2,3]],
                 [[1,2],[3,4]]])

print('X shape:', X.shape, '\nK shape:', K.shape)

corr2d_multi_in(X, K)

X shape: (2, 3, 3) 
K shape: (2, 2, 2)


<tf.Tensor: id=290, shape=(2, 2), dtype=float32, numpy=
array([[ 56.,  72.],
       [104., 120.]], dtype=float32)>

In [5]:
K[0]

<tf.Tensor: id=294, shape=(2, 2), dtype=int32, numpy=
array([[0, 1],
       [2, 3]])>